In [1]:
import os

root_dir = os.path.abspath('..')
data_dir = os.path.join(root_dir, 'data')
save_path = os.path.join(data_dir, 'assets_info.csv')

In [2]:
import pandas as pd
import numpy as np

## Sliding Window & Target

It's pretty interesting that market works not on all days (weekends & holidays).

Also, it works from 9am to 4pm (Eastern Time).


For this reason, I will consider all the available days as continuous.

And, in the day, I'll have 7 hours of data (from 9am to 4pm).

In [3]:
data = pd.read_csv(save_path)

In [4]:
def create_sliding_windows(data, window_size, future_offset):
    """
    Create sliding windows and corresponding targets from time series data.
    
    Parameters:
    - data: np.array, time series data (e.g., closing prices).
    - window_size: int, the number of points to include in each window (e.g., 30 days * 24 hours = 720).
    - future_offset: int, how many points ahead to select the target value (e.g., 24 for 24 hours later).
    
    Returns:
    - X_windows: np.array, 2D array where each row is a window of size `window_size`.
    - y_targets: np.array, 1D array of target values corresponding to each window.
    """
    
    # Initialize lists to store windows and targets
    X_windows = []
    y_targets = []
    
    # Sliding window loop
    for i in range(len(data) - window_size - future_offset):
        # Extract the window of `window_size` points
        window = data[i:i + window_size]
        
        # Extract the target value `future_offset` points after the window ends
        target = data[i + window_size + future_offset - 1]
        
        # Append the window and target to their respective lists
        X_windows.append(window)
        y_targets.append(target)
    
    # Convert the lists to numpy arrays
    X_windows = np.array(X_windows)
    y_targets = np.array(y_targets)
    
    return X_windows, y_targets


In [5]:
# Example time series data (close prices)
data_ = np.array([i for i in range(1000)])  # Replace with your actual time series data

# Set the parameters
window_size = 30 * 7  # 30 days * 7 hours = 210 points per window
future_offset = 7  # Target is the price 7 hours later

# Call the function
X_windows, y_targets = create_sliding_windows(data_, window_size, future_offset)

# Check the output
print(f'X_windows shape: {X_windows.shape}')  # Each row is a window of 720 points
print(f'y_targets shape: {y_targets.shape}')  # Corresponding targets

assert X_windows.shape[0] == y_targets.shape[0], 'Number of windows and targets do not match!'
assert X_windows.shape == (1000 - window_size - future_offset, window_size), 'Incorrect shape for X_windows!'

X_windows shape: (783, 210)
y_targets shape: (783,)


In [6]:
data.head(2)

,symbol,sector,industry,country,data
0,X,Basic Materials,Steel,United States,"{Timestamp('2024-03-04 09:30:00-0500', tz='Ame..."
1,HRL,Consumer Defensive,Packaged Foods,United States,"{Timestamp('2024-03-04 09:30:00-0500', tz='Ame..."


In [7]:
from pandas import Timestamp


FEATURE_COLS = ['symbol', 'sector', 'industry', 'country']
def get_features(data_row):
    return data_row[FEATURE_COLS].to_dict()

def get_values(data_row):
    data_str = data_row['data'] # str(dict)
    data_dict = eval(data_str)
    return np.array(list(data_dict.values()))

data_row_ = data.iloc[0]
features = get_features(data_row_)
values = get_values(data_row_)
features, values[:3]

({'symbol': 'X',
  'sector': 'Basic Materials',
  'industry': 'Steel',
  'country': 'United States'},
 array([47.45000076, 47.45500183, 47.77999878]))

In [8]:
from copy import deepcopy

def row_to_samples(data_row, window_size, future_offset):
    features = get_features(data_row)
    values = get_values(data_row)
    X_windows, y_targets = create_sliding_windows(values, window_size, future_offset)
    for i in range(len(X_windows)):
        sample = deepcopy(features)
        sample.update({f'value_{j}': X_windows[i][j] for j in range(window_size)})
        sample['target'] = y_targets[i]
        yield sample
    

# Set the parameters
window_size = 30 * 7  # 30 days * 7 hours = 210 points per window
future_offset = 7  # Target is the price 7 hours later

# Get the first row of data
data_row_ = data.iloc[0]
cnt = 10
for sample in row_to_samples(data_row_, window_size, future_offset):
    print(sample)
    break

{'symbol': 'X', 'sector': 'Basic Materials', 'industry': 'Steel', 'country': 'United States', 'value_0': np.float64(47.45000076293945), 'value_1': np.float64(47.45500183105469), 'value_2': np.float64(47.779998779296875), 'value_3': np.float64(47.814998626708984), 'value_4': np.float64(47.78499984741211), 'value_5': np.float64(47.755001068115234), 'value_6': np.float64(47.69499969482422), 'value_7': np.float64(47.650001525878906), 'value_8': np.float64(47.939998626708984), 'value_9': np.float64(48.005001068115234), 'value_10': np.float64(48.03499984741211), 'value_11': np.float64(48.07500076293945), 'value_12': np.float64(48.04499816894531), 'value_13': np.float64(47.7599983215332), 'value_14': np.float64(47.36000061035156), 'value_15': np.float64(47.38999938964844), 'value_16': np.float64(47.20000076293945), 'value_17': np.float64(47.1150016784668), 'value_18': np.float64(47.16999816894531), 'value_19': np.float64(47.229000091552734), 'value_20': np.float64(47.20000076293945), 'value_2

In [9]:
# data --> dataframe

def data_to_samples(data, window_size, future_offset):
    for _, data_row in data.iterrows():
        yield from row_to_samples(data_row, window_size, future_offset)

df = pd.DataFrame(data_to_samples(data, window_size, future_offset))

In [10]:
print(f'Number of samples: {len(df)}')

Number of samples: 127410


In [11]:
df.head(5)

,symbol,sector,industry,country,value_0,value_1,value_2,value_3,value_4,value_5,...,value_201,value_202,value_203,value_204,value_205,value_206,value_207,value_208,value_209,target
0,X,Basic Materials,Steel,United States,47.450001,47.455002,47.779999,47.814999,47.785000,47.755001,...,40.994999,41.340000,41.330002,41.240002,41.209999,40.907600,40.930000,40.724998,40.830002,40.299999
1,X,Basic Materials,Steel,United States,47.455002,47.779999,47.814999,47.785000,47.755001,47.695000,...,41.340000,41.330002,41.240002,41.209999,40.907600,40.930000,40.724998,40.830002,40.450001,40.599998
2,X,Basic Materials,Steel,United States,47.779999,47.814999,47.785000,47.755001,47.695000,47.650002,...,41.330002,41.240002,41.209999,40.907600,40.930000,40.724998,40.830002,40.450001,40.520000,40.500000
3,X,Basic Materials,Steel,United States,47.814999,47.785000,47.755001,47.695000,47.650002,47.939999,...,41.240002,41.209999,40.907600,40.930000,40.724998,40.830002,40.450001,40.520000,40.380001,40.134998
4,X,Basic Materials,Steel,United States,47.785000,47.755001,47.695000,47.650002,47.939999,48.005001,...,41.209999,40.907600,40.930000,40.724998,40.830002,40.450001,40.520000,40.380001,40.340000,40.209999


## Target

I will classify changes by more than `std / denominator` as 1 and -1, and the rest as 0. \
Denominator is chosen in such a way that the number of 1s and -1s is approximately equal to the number of 0s.


In [12]:
N_values = window_size

def get_change_over_std(data_row):
    """Return the target value for a given row of data."""
    std = np.std([data_row[f'value_{i}'] for i in range(N_values)])
    pre_last_price = data_row[f'value_{N_values-1}']
    last_price = data_row[f'target']
    return (last_price - pre_last_price) / std

DENOMINATOR = 4
def get_target(data_row):
    """Return the target value for a given row of data."""
    change_over_std = get_change_over_std(data_row)
    
    if abs(change_over_std) * DENOMINATOR < 1:
        return 0
    return 1 if change_over_std > 0 else -1

# df['target_class'] = df.apply(get_target, axis=1)
df.apply(get_change_over_std, axis=1).apply(abs).describe()

count    127410.000000
mean          0.431984
std           0.489741
min           0.000000
25%           0.129276
50%           0.294078
75%           0.568306
max           9.862311
dtype: float64

In [13]:
df['target_class'] = df.apply(get_target, axis=1)

In [14]:
df.target_class.value_counts()

target_class
 0    56380
 1    38230
-1    32800
Name: count, dtype: int64

## Additional preprocessing

Before moving to clustering and adding time-series based features, I will preprocess the data a bit:

* OHE to `sector`, `industry`, `country` columns
* Split the data into train and test (test = 0.1)
Motivation: I want in test to be samples from the assets model has never seen before.
* Scale the data


#### Splitting the data

In [15]:
from sklearn.model_selection import train_test_split

test_split = 0.15

df_train, df_test = train_test_split(df, test_size=test_split, random_state=42)

In [16]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#### OHE

In [17]:
ohe_cols = ['sector', 'industry', 'country']

# count values
for col in ohe_cols:
    print(col, df_train[col].nunique())

sector 11
industry 89
country 19


In [18]:
df_train['industry'].value_counts()

industry
Asset Management                    12029
Oil & Gas Equipment & Services       3944
Banks - Regional                     3282
Oil & Gas Midstream                  3204
REIT - Retail                        2864
                                    ...  
Telecom Services                      540
Other Industrial Metals & Mining      539
Industrial Distribution               388
Mortgage Finance                      355
Pharmaceutical Retailers              241
Name: count, Length: 89, dtype: int64

In [19]:
industry_size = df_train.groupby('industry').size().sort_values(ascending=False)
small_industries = industry_size[industry_size < 850].index
print("Amount of small industries:", len(small_industries))

df_train.loc[df_train.industry.isin(small_industries), 'industry'] = "default_industry"
df_test.loc[df_test.industry.isin(small_industries), 'industry'] = "default_industry"

Amount of small industries: 44


In [20]:
ohe_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
ohe_encoder.fit(df_train[ohe_cols])

df_train_ohe = pd.DataFrame(ohe_encoder.transform(df_train[ohe_cols]), columns=ohe_encoder.get_feature_names_out(ohe_cols))
df_test_ohe = pd.DataFrame(ohe_encoder.transform(df_test[ohe_cols]), columns=ohe_encoder.get_feature_names_out(ohe_cols))

In [21]:
df_train_ohe.shape

(108298, 76)

In [22]:
df_train_ohe.head(2)

,sector_Basic Materials,sector_Communication Services,sector_Consumer Cyclical,sector_Consumer Defensive,sector_Energy,sector_Financial Services,sector_Healthcare,sector_Industrials,sector_Real Estate,sector_Technology,...,country_India,country_Ireland,country_Israel,country_Japan,country_Luxembourg,country_Netherlands,country_Philippines,country_South Korea,country_United Kingdom,country_United States
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


#### Scaling

In [24]:
from tslearn.preprocessing import TimeSeriesScalerMeanVariance

scale_cols = [f'value_{i}' for i in range(N_values)]

X_train = df_train[scale_cols].values
X_test = df_test[scale_cols].values

# scale along with axis=1
scaler = TimeSeriesScalerMeanVariance(mu=0., std=1.)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

c:\Users\rodio\Desktop\Programming\ML\market_forecasting\.venv\Lib\site-packages\tslearn\bases\bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [34]:
random_row = np.random.randint(0, len(X_train))
n_checks = 25
for i in range(n_checks):
    random_row = np.random.randint(0, len(X_train))
    assert np.isclose(X_train_scaled[random_row].mean(), 0, atol=1e-5), 'Mean of the time series is not close to 0!'

    random_row = np.random.randint(0, len(X_test_scaled))
    assert np.isclose(X_test_scaled[random_row].mean(), 0, atol=1e-5), 'Mean of the time series is not close to 0!'

In [36]:
df_train[scale_cols] = X_train_scaled.reshape(-1, N_values)
df_test[scale_cols] = X_test_scaled.reshape(-1, N_values)

#### Final dataset

In [37]:
df_train.head(2)

,symbol,sector,industry,country,value_0,value_1,value_2,value_3,value_4,value_5,...,value_202,value_203,value_204,value_205,value_206,value_207,value_208,value_209,target,target_class
10029,XYF,Financial Services,Credit Services,China,0.298922,0.433164,0.282143,0.500285,0.500285,0.600968,...,-0.540092,-0.573652,-0.808577,-0.238382,-0.875698,-0.851534,-0.842137,-1.043500,3.955,1
28260,TWI,Industrials,Farm & Heavy Construction Machinery,United States,1.345733,1.391228,1.095743,0.947659,0.868045,0.924912,...,1.550457,1.504964,1.595952,1.857543,1.903036,1.823421,1.891664,1.857543,8.480,1


In [38]:
drop_cols = ohe_cols + ['symbol', 'target']
target_col = 'target_class'

y_train = df_train[target_col]
y_test = df_test[target_col]

X_train = df_train.drop(columns=drop_cols + [target_col])
X_test = df_test.drop(columns=drop_cols + [target_col])

X_simple_features_train = df_train_ohe
X_simple_features_test = df_test_ohe

In [39]:
print(X_train.shape, X_test.shape)

(108298, 210) (19112, 210)


In [40]:
# save
import os

X_train.to_csv(os.path.join(data_dir, 'X_train.csv'), index=False)
X_test.to_csv(os.path.join(data_dir, 'X_test.csv'), index=False)

X_simple_features_train.to_csv(os.path.join(data_dir, 'X_simple_features_train.csv'), index=False)
X_simple_features_test.to_csv(os.path.join(data_dir, 'X_simple_features_test.csv'), index=False)

y_train.to_csv(os.path.join(data_dir, 'y_train.csv'), index=False)
y_test.to_csv(os.path.join(data_dir, 'y_test.csv'), index=False)